**Saber cómo rebanar secuencias**

* Python incluye una sintaxis para dividir secuencias en piezas. El rebanado te permite acceder a un subconjunto de elementos de una secuencia con un esfuerzo mínimo. Los usos más simples para el rebanado son los tipos integrados list, str y bytes. El rebanado se puede extender a cualquier clase de Python que implemente los métodos especiales getitem y setitem

In [14]:
a = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h']
print('Middle two:  ', a[3:5])
print('All but ends:', a[1:7])

Middle two:   ['d', 'e']
All but ends: ['b', 'c', 'd', 'e', 'f', 'g']


Evita usar pasos y rebanado en una sola expresión

In [15]:
a[::2]

['a', 'c', 'e', 'g']

* Pero es posible utilizar múltiples expresiones con asterisco en una declaración de asignación de desempaquetado, siempre y cuando sean para capturar diferentes partes de la estructura multinivel que se está desempaquetando. 

* No recomiendo hacer lo siguiente (ver el Ítem 19: "Nunca desempaquetes más de tres variables cuando las funciones devuelvan múltiples valores" para orientación relacionada), pero entenderlo debería ayudarte a desarrollar una intuición sobre cómo se pueden usar las expresiones con asterisco en las asignaciones de desempaquetado.

In [16]:
car_inventory = {
 'Downtown': ('Silver Shadow', 'Pinto', 'DMC'),
 'Airport': ('Skyline', 'Viper', 'Gremlin', 'Nova'),
 }

In [17]:
car_inventory.items()

dict_items([('Downtown', ('Silver Shadow', 'Pinto', 'DMC')), ('Airport', ('Skyline', 'Viper', 'Gremlin', 'Nova'))])

In [18]:
((loc1, (best1, *rest1)),
 (loc2, (best2, *rest2))) = car_inventory.items()


In [19]:
#Se desempaqueta los elementos key (loc1 y best1, el resto de la tupla con *)
((loc1,(best1, *rest1)))

('Downtown', ('Silver Shadow', 'Pinto', 'DMC'))

In [20]:
((loc2,(best2, *rest2)))

('Airport', ('Skyline', 'Viper', 'Gremlin', 'Nova'))

In [21]:
((loc1, (best1, *rest1)),
(loc2, (best2, *rest2))) = car_inventory.items()


print(f'Best at {loc1} is {best1}, {len(rest1)} others')
print(f'Best at {loc2} is {best2}, {len(rest2)} others')

Best at Downtown is Silver Shadow, 2 others
Best at Airport is Skyline, 3 others


**Importante**

* Las expresiones con asterisco se convierten en instancias de listas en todos los casos. 

* Si no quedan elementos adicionales de la secuencia que se está desempaquetando, la parte de captura de todo será una lista vacía. Esto es especialmente útil cuando estás procesando una secuencia que sabes de antemano que tiene al menos N elementos:

In [22]:
short_list = [1,2,3,4,5,6]
first, second, *rest = short_list
print(first, second, rest)

1 2 [3, 4, 5, 6]


Pero con la adición de expresiones con asterisco, el valor de desempaquetar iteradores se hace evidente. Por ejemplo, aquí tengo un generador que produce las filas de un archivo CSV que contiene todos los pedidos de autos del concesionario de esta semana:

In [23]:
def generate_csv():
    yield ('Date', 'Make' , 'Model', 'Year', 'Price')

In [24]:
def generate_csv():
    yield ('Date', 'Make' , 'Model', 'Year', 'Price')
    yield ('2023-01-01', 'Toyota', 'Camry', 2022, 30000)
    yield ('2023-02-01', 'Honda', 'Accord', 2023, 32000)
    yield ('2023-03-01', 'Ford', 'Mustang', 2021, 45000)

In [25]:
# Obtener todas las filas generadas en una lista
all_csv_rows = list(generate_csv())

# Separar el encabezado y las filas
header = all_csv_rows[0]
rows = all_csv_rows[1:]

print('CSV Header:', header)
print('Row count: ', len(rows))

CSV Header: ('Date', 'Make', 'Model', 'Year', 'Price')
Row count:  3


In [26]:
import csv

def generate_csv():
    yield ('2023-01-01', 'Toyota', 'Camry', 2022, 30000)
    yield ('2023-02-01', 'Honda', 'Accord', 2023, 32000)
    yield ('2023-03-01', 'Ford', 'Mustang', 2021, 45000)

# Nombre del archivo CSV
csv_file = 'ejemplo.csv'

# Escribir los datos en el archivo CSV
with open(csv_file, 'w', newline='') as file:
    writer = csv.writer(file)
    
    # Escribir el encabezado
    writer.writerow(['Date', 'Make', 'Model', 'Year', 'Price'])
    
    # Escribir las filas de datos
    for row in generate_csv():
        writer.writerow(row)

print(f'Se ha creado el archivo CSV "{csv_file}" con los datos generados.')


Se ha creado el archivo CSV "ejemplo.csv" con los datos generados.


**Item 12:  Avoid Striding and Slicing in a Single Expression**

In [28]:
x = ['red', 'orange', 'yellow', 'green', 'blue', 'purple']

In [30]:
odds = x[::2]
print(odds)

['red', 'yellow', 'blue']


In [33]:
evens = x[1::2]
print(evens)

['orange', 'green', 'purple']


**Ítem 13: Prefiere el Desempaquetado de Captura General sobre el Slicing**

* Una limitación del desempaquetado básico (ver Ítem 6: "Prefiere el Desempaquetado de Asignación Múltiple sobre el Indexado") es que debes conocer la longitud de las secuencias que estás desempaquetando de antemano. 

* Por ejemplo, aquí tengo una lista de las edades de los autos que se están entregando en un concesionario. Cuando intento tomar los dos primeros elementos de la lista con desempaquetado básico, se genera una excepción en tiempo de ejecución:

In [39]:
car_ages = [0, 9, 4, 8, 7, 20, 19, 1, 6, 15]
car_ages_descending = sorted(car_ages,reverse=True)
print(car_ages_descending)

oldest = car_ages_descending[0]
second_oldest = car_ages_descending[1]
others = car_ages_descending[2:]

print(oldest,second_oldest,others)

[20, 19, 15, 9, 8, 7, 6, 4, 1, 0]
20 19 [15, 9, 8, 7, 6, 4, 1, 0]


* Este método funciona, pero todo el indexado y el slicing son visiblemente ruidosos. En la práctica, también es propenso a errores dividir los miembros de una secuencia en varios subconjuntos de esta manera porque es mucho más probable cometer errores de desplazamiento por uno; por ejemplo, podrías cambiar los límites en una línea y olvidar actualizar los demás.

* Para manejar mejor esta situación, **Python también admite el desempaquetado de captura general a través de una expresión con asterisco**. Esta sintaxis permite que una parte de la asignación de desempaquetado reciba todos los valores que no coincidieron con ninguna otra parte del patrón de desempaquetado. Aquí, uso una expresión con asterisco para lograr el mismo resultado que arriba sin indexado ni slicing:

In [40]:
oldest, second_oldest, *others = car_ages_descending
print(oldest,second_oldest,others)

20 19 [15, 9, 8, 7, 6, 4, 1, 0]



* Este código es más corto, más fácil de leer y ya no tiene la fragilidad propensa a errores de los índices de límites que deben mantenerse sincronizados entre líneas.

* **Una expresión con asterisco puede aparecer en cualquier posición, por lo que puedes obtener los beneficios del desempaquetado de captura general cada vez que necesites extraer una sección:**

In [51]:
oldest = car_ages_descending[0]
second_youngest = car_ages_descending[-1]
youngest = car_ages_descending[-2]
print(oldest,youngest,others)

print("")
*others, second_youngest, youngest = car_ages_descending
print(youngest,second_youngest,others)

20 1 [20, 19, 15, 9, 8, 7, 6, 4]

0 1 [20, 19, 15, 9, 8, 7, 6, 4]


**Ejercicio**

In [73]:
autos = {
    'auto1': ('Toyota','Corolla', 2023),
    'auto2': ('Honda','Civic',2022),
    }


In [78]:
(
    (key1, (marca, *rest)),
    (key2, (marca2, *rest2))
    
    ) = autos.items()

print(f'Best at {key1} is {marca}, {rest} others')
print(f'Best at {key2} is {marca2}, {rest2} others')

Best at auto1 is Toyota, ['Corolla', 2023] others
Best at auto2 is Honda, ['Civic', 2022] others


**Desempaquetado de diccionarios**

In [80]:
# Definición de un diccionario
auto = {
    'marca': 'Toyota',
    'modelo': 'Corolla',
    'año': 2023,
    'color': 'Rojo'
}

In [83]:
auto.items()

dict_items([('marca', 'Toyota'), ('modelo', 'Corolla'), ('año', 2023), ('color', 'Rojo')])

In [82]:
auto.values()

dict_values(['Toyota', 'Corolla', 2023, 'Rojo'])

In [81]:
# Desempaquetando el diccionario
marca, modelo, año, color = auto.values()

# Imprimiendo los valores desempaquetados
print(f"Marca: {marca}")
print(f"Modelo: {modelo}")
print(f"Año: {año}")
print(f"Color: {color}")


Marca: Toyota
Modelo: Corolla
Año: 2023
Color: Rojo


In [84]:
def imprimir_auto(marca, modelo, año, color):
    print(f"Marca: {marca}, Modelo: {modelo}, Año: {año}, Color: {color}")

# Llamando a la función con desempaquetado de diccionarios
imprimir_auto(**auto)


Marca: Toyota, Modelo: Corolla, Año: 2023, Color: Rojo


In [86]:
it = iter(range(1,3))
first, second = it
print(f"{first} and {second}")

1 and 2


**Item 14: Ordenar por Criterios Complejos Utilizando el Parámetro key**

* El tipo integrado list proporciona un método sort para ordenar los elementos de una instancia de lista basándose en diversos criterios. Por defecto, sort ordenará el contenido de una lista en orden ascendente natural de los elementos. Por ejemplo, aquí ordeno una lista de enteros de menor a mayor:

In [87]:
numbers = [93,86,11,68,70]
numbers.sort()
print(numbers)

[11, 68, 70, 86, 93]


77